In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

### Data Loading

In [2]:
# Loads the IMDB Movie review dataset
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [3]:
# Captures the train and test data
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

# Creates numpy arrays from the train and test labels
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

### Data Preprocessing

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
vocab_size = 10000
# Embedding dimensions
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


# Defines the tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
# Fits the tokenizer
tokenizer.fit_on_texts(training_sentences)

# Creates a word index from the tokenizer
word_index = tokenizer.word_index

# Generates the sequences from the sentences using the tokenizer
sequences = tokenizer.texts_to_sequences(training_sentences)
# Pads the sequences
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Creates test sequences using tokenizer and padding
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [6]:
# Creates a reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


### Model Preparation

In [7]:
# Model Architecture
model = tf.keras.Sequential([
    # Embedding layer
    tf.keras.layers.Embedding(vocab_size, 
                              embedding_dim, 
                              input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiles the model
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


### Model Fitting

In [9]:
# Sets the number of epochs for training
num_epochs = 10

# Model fitting
model.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, 
                           testing_labels_final
                          ))

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1083 - accuracy: 0.9707 - val_loss: 0.4389 - val_accuracy: 0.8292
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0319 - accuracy: 0.9952 - val_loss: 0.5184 - val_accuracy: 0.8291
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0120 - accuracy: 0.9984 - val_loss: 0.5699 - val_accuracy: 0.8272
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0055 - accuracy: 0.9993 - val_loss: 0.6742 - val_accuracy: 0.8210
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.6966 - val_accuracy: 0.8244
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 9.9060e-04 - accuracy: 0.9999 - val_loss: 0.7437 - val_accuracy: 0.8228
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 4.1758e-04 - accuracy: 1.0000 - val_loss: 0.7810 - val_accuracy: 0.825

In [11]:
e = model.layers[0]
weights = e.get_weights()[0]
# Shape: (vocab_size, embedding_dim)
print(weights.shape)

(10000, 16)


In [12]:
import io

In [13]:
out_v = io.open('./data/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('./data/meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [14]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
